# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [8]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [9]:
!cat talks.csv

title,type,url_slug,venue,date,location,talk_url,description
Quantum Estimation in Strong Fields: in situ ponderomotive sensing,Conference,Photon2020,Photon 2020,2020-09-01,Virtual Conference,https://youtu.be/dRmk8-vE6ls,The classical and quantum Fisher information is derived for the strong field approximation to enable optimal in situ  measurements of laser intensity.
Battle 3: Numerical vs Analytical Methods,Invited Talk,QBat2020,Quantum Battles in Attoscience,2020-07-03,Virtual Conference,https://www.youtube.com/watch?v=VJnFfHVDym4&ab_channel=QuantumBattles,"A panel discussion in a debate format on the pros and cons of ab-initio, numerical and analytical methods."
The Spiral-like Structure in Strong-Field Ionization: Unwinding Holographic Interference,Conference,DAMOP2020, APS Division of Atomic Molecular and Optical Physics (DAMOP) 2020,2020-06-01,Virtual Conference,http://meetings.aps.org/Meeting/DAMOP20/Session/N04.6,A new interference structure presented for photoelectron hologr

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [10]:
talks = pd.read_csv("talks.csv", sep=",", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Quantum Estimation in Strong Fields: in situ p...,Conference,Photon2020,Photon 2020,2020-09-01,Virtual Conference,https://youtu.be/dRmk8-vE6ls,The classical and quantum Fisher information i...
1,Battle 3: Numerical vs Analytical Methods,Invited Talk,QBat2020,Quantum Battles in Attoscience,2020-07-03,Virtual Conference,https://www.youtube.com/watch?v=VJnFfHVDym4&ab...,A panel discussion in a debate format on the p...
2,The Spiral-like Structure in Strong-Field Ioni...,Conference,DAMOP2020,APS Division of Atomic Molecular and Optical ...,2020-06-01,Virtual Conference,http://meetings.aps.org/Meeting/DAMOP20/Sessio...,A new interference structure presented for pho...
3,Python in Research: Quantum Sensing in Strong ...,Tutorial,PythonTalk,University College London,2019-04-25,"London, United Kingdom",NaN,"Python class drop in session, example of how p..."
4,Treating Branch Cuts in Quantum Trajectory Mod...,Invited Talk,ICFOTalk1,Instituto de Ciencias Fotonicas (ICFO),2019-04-25,"Barcelona, Spain",NaN,"Presenting edited version of the CQSFA, which ..."
5,Holography and Electron Correlation in Strong ...,Invited Talk,WuhanTalk,"WIPM, Chinese Academy of Science",2018-11-13,"Wuhan, China",NaN,Presentation of my PhD thesis work.
6,Treating Branch Cuts in Quantum Trajectory Mod...,Conference,LPhys2018,"Annual International Laser Physics Workshop, L...",2018-07-16,"University of Nottingham, United Kingdom",NaN,"Presenting edited version of the CQSFA, which ..."
7,Quantum Interference in Below-Threshold Nonseq...,Conference,Atto-FEL,Atto-FEL,2018-07-02,"University College London, United Kingdom",NaN,Demonstration of interferference effects prese...
8,Coulomb-distorted quantum orbits in photoelect...,Conference,AMIG2018,Atomic and Molecular Interactions Group (AMIG),2018-06-21,"University College London, United Kingdom",NaN,Presenting the development of the Coulomb corr...
9,Quantum Interference in Below-Threshold Nonseq...,Invited Talk,ICFO2,Instituto de Ciencias Fotonicas (ICFO),2018-05-22,"Barcelona, Spain",NaN,Demonstration of interferference effects prese...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [11]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [12]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[See talk here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [13]:
!ls ../_talks

2016-04-04-WG1.md	 2018-07-02-Atto-FEL.md   2019-04-25-PythonTalk.md
2016-07-11-LPhys2018.md  2018-07-16-LPhys2018.md  2020-06-01-DAMOP2020.md
2018-05-22-ICFO2.md	 2018-11-13-WuhanTalk.md  2020-07-03-QBat2020.md
2018-06-21-AMIG2018.md	 2019-04-25-ICFOTalk1.md  2020-09-01-Photon2020.md


In [14]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
